# ANN training

In [6]:
import os
import sys

from ase import Atoms, Atom, units
import ase.io
from ase.calculators.emt import EMT
from ase.build import add_adsorbate, fcc100, fcc110, fcc111
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md import VelocityVerlet
from ase.constraints import FixAtoms
from ase.md import MDLogger
from ase.md.langevin import Langevin

from amp import Amp
from amp.descriptor.gaussian import Gaussian
from amp.model.neuralnetwork import NeuralNetwork
from amp.model import LossFunction

print("You are all set for the tutorial.")

You are all set for the tutorial.


# ANN training #2: training on MD + NEB

In [7]:
!rm -rf amp-*

In [9]:
! OMP_NUM_THREADS=1 OPENBLAS_NUM_THREADS=1 MKL_NUM_THREADS=1 PYTHONUNBUFFERED=1 "{sys.executable}" train_amp_2.py

In [10]:
!mv amp.amp amp_2.amp

### MD with ANN potential (amp_2.amp) start with hollow site

In [9]:
slab = fcc100('Al', size=(3, 3, 4))
add_adsorbate(slab, 'Au', 1.7, 'hollow')
slab.center(axis=2, vacuum=6.0)

# Fix third and fourth layers
constraint = FixAtoms(mask=[atom.tag > 2 for atom in slab])
slab.set_constraint(constraint)

# Enarlge slab to test fast ANN-MD
supercell = slab.repeat((1, 1, 1))

# Set the trained Amp as calculator
calc = Amp.load('amp_2.amp')
supercell.calc = calc

# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(supercell, temperature_K=300)

# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(supercell, 2 * units.fs)  # 2 fs time step.
# dyn = Langevin(supercell, 2 * units.fs, temperature_K=300, friction=0.002)

# traj 파일 열기 (쓰기 모드, atoms 초기 상태 등록)
with ase.io.Trajectory('hollow.traj', 'w', supercell) as traj:
    # 5 스텝마다 원자 구조 저장
    dyn.attach(traj.write, interval=5)

    # 로그도 파일에 기록 (옵션)
    dyn.attach(MDLogger(dyn, supercell, 'hollow.log', header=True,
                        stress=False, peratom=False), interval=5)

    # 짧게 예열
    dyn.run(100)
    # 안정하면 늘리기
    dyn.run(900)

In [ ]:
images = ase.io.read('training_set_2.traj', ':')
ase.io.write('training_set_2.traj', images[:10])

In [10]:
images = ase.io.read('hollow.traj', ':')
ase.io.write('hollow.extxyz', images)

/Users/inwon/anaconda3/lib/python3.11/site-packages/ase/io/extxyz.py:318: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


### MD with ANN potential (amp_2.amp) start with bridge site

In [ ]:
slab = fcc100('Al', size=(3, 3, 4))
add_adsorbate(slab, 'Au', 1.7, 'bridge')
slab.center(axis=2, vacuum=6.0)

# Fix third and fourth layers
constraint = FixAtoms(mask=[atom.tag > 2 for atom in slab])
slab.set_constraint(constraint)

# Enarlge slab to test fast ANN-MD
supercell = slab.repeat((1, 1, 1))

# Set the trained Amp as calculator
calc = Amp.load('amp_2.amp')
supercell.calc = calc

# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(supercell, temperature_K=300)

# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(supercell, 2 * units.fs)  # 2 fs time step.
# dyn = Langevin(supercell, 2 * units.fs, temperature_K=300, friction=0.002)

# traj 파일 열기 (쓰기 모드, atoms 초기 상태 등록)
with ase.io.Trajectory('bridge.traj', 'w', supercell) as traj:
    # 5 스텝마다 원자 구조 저장
    dyn.attach(traj.write, interval=5)

    # 로그도 파일에 기록 (옵션)
    dyn.attach(MDLogger(dyn, supercell, 'bridge.log', header=True,
                        stress=False, peratom=False), interval=5)

    # 짧게 예열
    dyn.run(100)
    # 안정하면 늘리기
    dyn.run(900)

In [ ]:
images = ase.io.read('bridge.traj', ':')
ase.io.write('bridge.extxyz', images)